# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments          Customer  \
0  Mar 15 2023 12:44PM  259380        19      9248106  Eywa Pharma Inc.   
1  Mar 15 2023 12:44PM  259380        19      9265137  Eywa Pharma Inc.   
2  Mar 15 2023 12:44PM  259380        19      9265752  Eywa Pharma Inc.   
3  Mar 15 2023 12:14PM  259379        18  SEQ-9268441           Seqirus   
4  Mar 15 2023 12:08PM  259378        10     HHHH9126         Hush Hush   
5  Mar 15 2023 12:04PM  259375        10    MSP219537  Methapharm, Inc.   
6  Mar 15 2023 12:04PM  259375        10    MSP219538  Methapharm, Inc.   
7  Mar 15 2023 12:04PM  259375        10    MSP219539  Methapharm, Inc.   
8  Mar 15 2023 12:04PM  259375        10    MSP219540  Methapharm, Inc.   
9  Mar 15 2023 12:04PM  259375        10    MSP219541  Methapharm, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
22  259375       Released         12
23  259376       Released          2
24  259378       Released          1
25  259379       Released          1
26  259380       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259375                NaN        NaN      12.0
259376                NaN        NaN       2.0
259378                NaN        NaN       1.0
259379                NaN        NaN       1.0
259380                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259190               10.0       27.0       6.0
259255                6.0       14.0       1.0
259291                0.0        2.0       4.0
259312                0.0        1.0       0.0
259319                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259190                 10         27         6
259255                  6         14         1
259291                  0          2         4
259312                  0          1         0
259319                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259190         10         27         6
1               259255          6         14         1
2               259291          0          2         4
3               259312          0          1         0
4               259319          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259190        10        27        6
1               259255         6        14        1
2               259291                   2        4
3               259312                   1         
4               259319                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse           Customer
0   Mar 15 2023 12:44PM  259380        19   Eywa Pharma Inc.
3   Mar 15 2023 12:14PM  259379        18            Seqirus
4   Mar 15 2023 12:08PM  259378        10          Hush Hush
5   Mar 15 2023 12:04PM  259375        10   Methapharm, Inc.
17  Mar 15 2023 12:04PM  259376        10        Emerginnova
19  Mar 15 2023 12:02PM  259374        10   Methapharm, Inc.
20  Mar 15 2023 11:47AM  259373        10     CLINUVEL, Inc.
23  Mar 15 2023 11:07AM  259371        21  NBTY Global, Inc.
24  Mar 15 2023 10:46AM  259368        10   Eywa Pharma Inc.
31  Mar 15 2023 10:17AM  259362        10     CLINUVEL, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse           Customer Completed  \
0  Mar 15 2023 12:44PM  259380        19   Eywa Pharma Inc.             
1  Mar 15 2023 12:14PM  259379        18            Seqirus             
2  Mar 15 2023 12:08PM  259378        10          Hush Hush             
3  Mar 15 2023 12:04PM  259375        10   Methapharm, Inc.             
4  Mar 15 2023 12:04PM  259376        10        Emerginnova             
5  Mar 15 2023 12:02PM  259374        10   Methapharm, Inc.             
6  Mar 15 2023 11:47AM  259373        10     CLINUVEL, Inc.             
7  Mar 15 2023 11:07AM  259371        21  NBTY Global, Inc.             
8  Mar 15 2023 10:46AM  259368        10   Eywa Pharma Inc.         1   
9  Mar 15 2023 10:17AM  259362        10     CLINUVEL, Inc.             

  Executing Released  
0                  3  
1                  1  
2                  1  
3                 12  
4                  2  
5                  1  
6                  3  
7         1           
8         6           
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse           Customer Released  \
0  Mar 15 2023 12:44PM  259380        19   Eywa Pharma Inc.        3   
1  Mar 15 2023 12:14PM  259379        18            Seqirus        1   
2  Mar 15 2023 12:08PM  259378        10          Hush Hush        1   
3  Mar 15 2023 12:04PM  259375        10   Methapharm, Inc.       12   
4  Mar 15 2023 12:04PM  259376        10        Emerginnova        2   
5  Mar 15 2023 12:02PM  259374        10   Methapharm, Inc.        1   
6  Mar 15 2023 11:47AM  259373        10     CLINUVEL, Inc.        3   
7  Mar 15 2023 11:07AM  259371        21  NBTY Global, Inc.            
8  Mar 15 2023 10:46AM  259368        10   Eywa Pharma Inc.            
9  Mar 15 2023 10:17AM  259362        10     CLINUVEL, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7         1            
8         6         1  
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse          Customer Released Executing  \
0  Mar 15 2023 12:44PM  259380        19  Eywa Pharma Inc.        3             
1  Mar 15 2023 12:14PM  259379        18           Seqirus        1             
2  Mar 15 2023 12:08PM  259378        10         Hush Hush        1             
3  Mar 15 2023 12:04PM  259375        10  Methapharm, Inc.       12             
4  Mar 15 2023 12:04PM  259376        10       Emerginnova        2             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Mar 15 2023 12:44PM  259380        19  Eywa Pharma Inc.       3.0   
1  Mar 15 2023 12:14PM  259379        18           Seqirus       1.0   
2  Mar 15 2023 12:08PM  259378        10         Hush Hush       1.0   
3  Mar 15 2023 12:04PM  259375        10  Methapharm, Inc.      12.0   
4  Mar 15 2023 12:04PM  259376        10       Emerginnova       2.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse          Customer  Released  \
0  Mar 15 2023 12:44PM  259380        19  Eywa Pharma Inc.       3.0   
1  Mar 15 2023 12:14PM  259379        18           Seqirus       1.0   
2  Mar 15 2023 12:08PM  259378        10         Hush Hush       1.0   
3  Mar 15 2023 12:04PM  259375        10  Methapharm, Inc.      12.0   
4  Mar 15 2023 12:04PM  259376        10       Emerginnova       2.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2593636      32.0       10.0        1.0
12          259312       0.0        1.0        0.0
15          777736      11.0       43.0       16.0
18          259379       1.0        0.0        0.0
19          518740      17.0        8.0        0.0
20          259354       1.0        0.0        0.0
21          259371       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2593636      32.0       10.0        1.0
1        12   259312       0.0        1.0        0.0
2        15   777736      11.0       43.0       16.0
3        18   259379       1.0        0.0        0.0
4        19   518740      17.0        8.0        0.0
5        20   259354       1.0        0.0        0.0
6        21   259371       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      32.0       10.0        1.0
1        12       0.0        1.0        0.0
2        15      11.0       43.0       16.0
3        18       1.0        0.0        0.0
4        19      17.0        8.0        0.0
5        20       1.0        0.0        0.0
6        21       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   32.0
1        12  Released    0.0
2        15  Released   11.0
3        18  Released    1.0
4        19  Released   17.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   18    19   20   21
Status                                         
Completed   1.0  0.0  16.0  0.0   0.0  0.0  0.0
Executing  10.0  1.0  43.0  0.0   8.0  0.0  1.0
Released   32.0  0.0  11.0  1.0  17.0  1.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   18    19   20   21
0          Completed   1.0  0.0  16.0  0.0   0.0  0.0  0.0
1          Executing  10.0  1.0  43.0  0.0   8.0  0.0  1.0
2           Released  32.0  0.0  11.0  1.0  17.0  1.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   18    19   20   21
0  Completed   1.0  0.0  16.0  0.0   0.0  0.0  0.0
1  Executing  10.0  1.0  43.0  0.0   8.0  0.0  1.0
2   Released  32.0  0.0  11.0  1.0  17.0  1.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()